# Steps to download Mistral LLM and link it to google colab


1.   Download the model into your local system. Please note you need have git and git-lfs installed on your system.

*   git lfs install
*   git clone https://huggingface.co/mistralai/Mistral-7B-v0.1

2.  Now, remove the .git folder from the downloaded folder path, which might be consuming a lot of GB.

3.  Upload the downloaded LLM model folder to your google drive- if the space is less on your GDrive, you may opt for google membership free of cost(google provides 3 months of trail pack which comes with 100 GBs of GDrive space)

4.  Make sure to change runtime type to GPU

5.  Link Google Drive to your colab notebook as follows



# Mount GDrive to Your Colab Notebook

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Other Alternative: Directly download Mistral Model from Huggingface website
#### Here the challenge is, if you want to experiment this on free google colab notebook, the notebook might crash(`which happened to me as well, so that's why I created a trick mentioned in above step`) since the model is huge and even to download the model you might require lot of RAM and Space.

# To Download model directly to Colab Workspace

In [ ]:
# !git lfs install
# !git clone https://huggingface.co/mistralai/Mistral-7B-v0.1

Git LFS initialized.
Cloning into 'Mistral-7B-v0.1'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 87 (delta 43), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (87/87), 473.17 KiB | 2.67 MiB/s, done.
Filtering content: 100% (5/5), 3.46 GiB | 3.15 MiB/s, done.
Encountered 4 file(s) that may not have been copied correctly on Windows:
	model-00002-of-00002.safetensors
	pytorch_model-00002-of-00002.bin
	model-00001-of-00002.safetensors
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


## Don't forget to remove .git folder to get rid of unwanted space consumed

In [ ]:
! rm -rf /content/Mistral-7B-v0.1/.git

# Install Required Packages

In [2]:
!pip install -q torch datasets
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                transformers==4.31.0 \
                trl==0.4.7?

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement trl==0.4.7? (from versions: 0.0.1, 0.0.2, 0.0.3, 0.1.0, 0.2.0, 0.2.1, 0.3.0, 0.3.1, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.5.0, 0.6.0, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.7.5, 0.7.6, 0.7.7)
ERROR: No matching distribution found for trl==0.4.7?


In [3]:
! pip install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00


# Check if the GPU is enabled?

In [5]:
!nvidia-smi

Thu Dec 28 18:45:26 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Import Packages

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

### Set the device to cuda, to Load model on GPUs `since we have a single GPU it is by default set to cuda:0`

In [2]:
torch.set_default_device('cuda')

## Load tokenizer and model from the linked google drive path

In [3]:
# Load model with 8 bit quantization, since it might crash with float16 precision.
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Generative AI/LLM Models/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Generative AI/LLM Models/Mistral-7B-v0.1/",
    trust_remote_code=True,
    device_map="auto",
    load_in_8bit=True,# For 8 bit quantization,
    max_memory={0:"15GB"}
)
model.eval()
model = torch.compile(model, mode = "max-autotune", backend="inductor")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
model.device

device(type='cuda', index=0)

## Inference Model: Question 1


In [16]:
text = "What is the capital of India?"

device = 'cuda'
model_inputs = tokenizer(text, return_tensors="pt").to(model.device)

generated_ids = model.generate(**model_inputs, temperature=0.1, top_k=1, top_p=1.0, repetition_penalty=1.4, min_new_tokens=16, max_new_tokens=128, do_sample=True)
decoded = tokenizer.decode(generated_ids[0])
print(decoded)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> What is the capital of India?
New Delhi. It was established as a temporary national capital in 1930 and became permanent after independence from Britain on August 2, 1846. The city has been home to many important events including Gandhi’s assassination (January 7th), Jawaharlal Nehru University riots(June-August)and more recently protests against corruption by Anna Hazare . In addition there are also several museums such as National Museum which houses artifacts dating back thousands years ago! There were even some rumors about an underground tunnel system connecting all these places together but no


## Clean the final LLM Generated Answer

In [15]:
answer = decoded.split(text)[-1]
answer = answer.replace("<s>", "").replace("</s>", "").replace("<pad>", "")
print(answer)


New Delhi. It was established as a temporary national capital in 1930 and became permanent after independence from Britain on August 2, 1846. The city has been home to many important events including Gandhi’s assassination (January 7th), Jawaharlal Nehru University riots(June-August)and more recently protests against corruption by Anna Hazare . In addition there are also several museums such as National Museum which houses artifacts dating back thousands years ago! There were even some rumors about an underground tunnel system connecting all these places together but no


## Inference Model: Question 2

In [17]:
text = "What is Generative AI, Can you please help me understand it's usecases?"

device = 'cuda'
model_inputs = tokenizer(text, return_tensors="pt").to(model.device)

generated_ids = model.generate(**model_inputs, temperature=0.1, top_k=1, top_p=1.0, repetition_penalty=1.4, min_new_tokens=16, max_new_tokens=128, do_sample=True)
decoded = tokenizer.decode(generated_ids[0])
print(decoded)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> What is Generative AI, Can you please help me understand it's usecases?
- 01 May 2023
- 4 Minutes to read

# Overview of ChatGPT and GTP-3.5 models for generative artificial intelligence (AI) applications in the healthcare industry: Use cases & benefits

Generating text using a large language model like OpenAI’s chatbot or other similar tools can be useful when writing content such as blog posts, articles, emails etc., but there are also many potential uses within medical settings where these types may come into play too! For example - if someone has trouble remembering what they said during an appointment with their doctor then


In [18]:
answer = decoded.split(text)[-1]
answer = answer.replace("<s>", "").replace("</s>", "").replace("<pad>", "")
print(answer)


- 01 May 2023
- 4 Minutes to read

# Overview of ChatGPT and GTP-3.5 models for generative artificial intelligence (AI) applications in the healthcare industry: Use cases & benefits

Generating text using a large language model like OpenAI’s chatbot or other similar tools can be useful when writing content such as blog posts, articles, emails etc., but there are also many potential uses within medical settings where these types may come into play too! For example - if someone has trouble remembering what they said during an appointment with their doctor then
